In [1]:
numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)] + ["day_number"]

fields = ["id", "label"] + numeric_features + categorical_features

In [2]:
import pandas as pd

train_name = '/home/users/datasets/criteo/criteo_train1'
df = pd.read_csv(train_name, sep='\t', names=fields, index_col = None)
df.head()

,id,label,if1,if2,if3,if4,if5,if6,if7,if8,...,cf18,cf19,cf20,cf21,cf22,cf23,cf24,cf25,cf26,day_number
0,0,0,NaN,600.0,1.0,342.0,9.0,0.0,0.0,18,...,b8170bba,9512c20b,9b6712fa,e611fb63,27fc20dc,NaN,201d7147,57e36578,b757e957,day_0
1,2,0,3.0,239.0,3.0,NaN,8.0,0.0,0.0,139,...,b8170bba,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,10c76e2c,30436bfc,e1be5ef2,day_0
2,4,0,3.0,124.0,39.0,398.0,3.0,0.0,0.0,1,...,a1eb1511,9512c20b,47849e55,73b3f46d,d994ba60,NaN,d3d5ac76,ff654802,b757e957,day_0
3,6,0,12.0,85.0,4.0,59.0,6.0,0.0,0.0,7,...,b8170bba,cc7a7d35,156cbe87,96fbe197,15562d5d,d3df7183,893704a5,ff654802,e1be5ef2,day_0
4,8,0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-1,...,b8170bba,9512c20b,c07b84a7,323fe50a,504569ef,NaN,1aa62c4a,0b3c06d0,2ccea557,day_0


In [29]:
df.nunique()

id            5014590
label               2
if1              5270
if2              7974
if3               431
if4              8706
if5              2244
if6               381
if7               229
if8              5622
if9               471
if10               13
if11              170
if12           217150
if13              815
cf1           1037908
cf2             21684
cf3             15051
cf4              7040
cf5             19033
cf6                 3
cf7              6601
cf8              1285
cf9                49
cf10           745502
cf11           122781
cf12            88663
cf13               10
cf14             2151
cf15             8220
cf16               60
cf17                3
cf18              940
cf19               14
cf20          1136924
cf21           388105
cf22           935193
cf23            89003
cf24             9711
cf25               74
cf26               33
day_number          7
dtype: int64

In [3]:
# оставляем только категориальные фичи, где разлчиных значений не больше unique_threshold
unique_threshold = 1000
cat_features_to_use = []
for el in dict(df.nunique()).items():
    if (el[1] < unique_threshold) & ('cf' in el[0]):
        cat_features_to_use.append(el[0])
cat_features_to_use        

['cf6', 'cf9', 'cf13', 'cf16', 'cf17', 'cf18', 'cf19', 'cf25', 'cf26']

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split, GridSearchCV

#
# Model pipeline
#

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
#    ,('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, cat_features_to_use)
    ]
)

# Now we have a full prediction pipeline.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    #('linearregression', LinearRegression())
    ('linearregression', RandomForestClassifier(max_depth=4, verbose=2, n_jobs=5, max_features=0.3))
])

In [31]:
# model = Pipeline(steps=[
#     ('preprocessor', SimpleImputer(strategy='median')),
#     ('linearregression', LinearRegression())
# ])

In [5]:
%%time
import os, sys
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump
from sklearn.metrics import log_loss


X_train, X_test, y_train, y_test = (df[df['day_number'] != 'day_6'][numeric_features + cat_features_to_use],
df[df['day_number'] == 'day_6'][numeric_features + cat_features_to_use],
df[df['day_number'] != 'day_6']['label'],
df[df['day_number'] == 'day_6']['label'])

#
# Train the model
#
model.fit(X_train, y_train)

model_score = log_loss(y_test.values, model.predict_proba(X_test))

logging.info(f"model score: {model_score:.3f}")

# save the model
dump(model, "{}.joblib".format(1))


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100


[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:  2.3min


building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77

[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  7.0min finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    2.6s finished


CPU times: user 34min 34s, sys: 26.1 s, total: 35min
Wall time: 7min 48s


['1.joblib']

In [6]:
model_score

0.13329860493474563